## 1. Set up the scraper
* Download data from nfl.com
* Use internal api with Bearer Key

### 1.1 Get a working example for Bearer token

In [2]:
import time
from browsermobproxy import Server
from selenium import webdriver
import json
import os
BMP_PATH = os.path.abspath('browsermob-proxy-2.1.4/bin/browsermob-proxy.bat')

# Start BrowserMob Proxy
server = Server(BMP_PATH)
server.start()
proxy = server.create_proxy()

# Configure Chrome with the proxy
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={proxy.proxy}')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# Navigate to the website
driver.get("https://nfl.com/scores")

# Start capturing network traffic
proxy.new_har("nfl", options={'captureHeaders': True})

# Wait for the header with the name "Authorization"
timeout = 60  # Adjust the timeout as needed
def get_auth_token():
    start_time = time.time()
    while time.time() - start_time < timeout:
        har = proxy.har
        for entry in har['log']['entries']:
            request_headers = entry['request']['headers']
            for header in request_headers:
                if header['name'] == 'Authorization':
                    return header['value']
        time.sleep(1)
    return None
auth_token = get_auth_token()

# Store the auth token
auth_path = 'auth.json'
auth_json = {
    'time': time.time(),
    'token': auth_token
}
with open(auth_path, 'w') as f:
    json.dump(auth_json, f)

# Stop capturing network traffic
har = proxy.har

# Close the browser and BrowserMob Proxy
driver.quit()
server.stop()

### 1.2 Create a class that manages requests to nfl.com

In [5]:
from nfl_client import NFLClient
client = NFLClient(headless=True)

In [7]:
client.load_auth_token(store_har=True)
print('Auth Token:', client.auth_token)

Auth Token: Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJjbGllbnRJZCI6ImU1MzVjN2MwLTgxN2YtNDc3Ni04OTkwLTU2NTU2ZjhiMTkyOCIsImNsaWVudEtleSI6IjRjRlVXNkRtd0pwelQ5TDdMckczcVJBY0FCRzVzMDRnIiwiaXNzIjoiTkZMIiwiZGV2aWNlSWQiOiI0YTIzZGM2My00YmQ2LTQ0OTMtYWVmNC0yNDk0OGRkNGMxYTUiLCJwbGFucyI6W3sicGxhbiI6ImZyZWUiLCJleHBpcmF0aW9uRGF0ZSI6IjIwMjQtMTEtMDQiLCJzb3VyY2UiOiJORkwiLCJzdGFydERhdGUiOiIyMDIzLTExLTA1Iiwic3RhdHVzIjoiQUNUSVZFIiwidHJpYWwiOmZhbHNlfV0sIkRpc3BsYXlOYW1lIjoiV0VCX0RFU0tUT1BfREVTS1RPUCIsIk5vdGVzIjoiIiwiZm9ybUZhY3RvciI6IkRFU0tUT1AiLCJsdXJhQXBwS2V5IjoiU1pzNTdkQkdSeGJMNzI4bFZwN0RZUSIsInBsYXRmb3JtIjoiREVTS1RPUCIsInByb2R1Y3ROYW1lIjoiV0VCIiwiY2l0eSI6ImFwcGxlIHZhbGxleSIsImNvdW50cnlDb2RlIjoiVVMiLCJkbWFDb2RlIjoiODAzIiwiaG1hVGVhbXMiOlsiMTA0MDI1MTAtODkzMS0wZDVmLTk4MTUtNzliYjc5NjQ5YTY1IiwiMTA0MDI1MjAtOTZiZi1lOWYyLTRmNjgtODUyMWNhODk2MDYwIiwiMTA0MDQ0MDAtM2IzNS0wNzNmLTE5N2UtMTk0YmI4MjQwNzIzIl0sInJlZ2lvbiI6IkNBIiwiYnJvd3NlciI6IkNocm9tZSBIZWFkbGVzcyIsImNlbGx1bGFyIjpmYWxzZSwiZW52aXJvbm1lbnQiOiJwcm9kdWN0aW9u

### 1.3 Prepare url endpoints into json version for openapi

In [16]:
from urllib.parse import urlparse, parse_qs
import re


id_re = re.compile(r"([0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12})")

def urlparse_json(url):
    r = urlparse(url)
    id_match = id_re.search(r.path)
    path = r.path
    query_params = {}
    for key, item in parse_qs(r.query).items():
        item_type = 'string'
        example = ''
        if len(item) > 1:
            item_type = 'array'
            example = ','.join(item)
        if len(item) == 1:
            example = item[0]
            try:
                example = float(item[0])
                item_type = 'number'
            except:
                pass
            try:
                example = int(item[0])
                item_type = 'integer'
            except:
                pass
            if item[0] in ['true', 'false']:
                item_type = 'boolean'
        query_params.update({
            key: {
                'type': item_type,
                'example': example
            }
        })
    path_params = {}
    if id_match:
        uuid = id_match.group(0)
        path = r.path.replace(uuid, '{id}')
        path_params = {'id': {'type': 'string', 'example': uuid}}
    return {
        path: {
            'query_parameters': query_params,
            'path_parameters': path_params
        }
    }

url = 'https://api.nfl.com/football/v2/games/b07eb396-f053-11ed-b4a7-bab79e4492fa?withExternalIds=true&int_item=4&num_item=3.2'
url_data = urlparse_json(url)
url_data

{'/football/v2/games/{id}': {'query_parameters': {'withExternalIds': {'type': 'boolean',
    'example': 'true'},
   'int_item': {'type': 'integer', 'example': 4},
   'num_item': {'type': 'number', 'example': 3.2}},
  'path_parameters': {'id': {'type': 'string',
    'example': 'b07eb396-f053-11ed-b4a7-bab79e4492fa'}}}}

#### 1.3.1 Create something to convert this into openapi

In [18]:
import yaml

def add_api_component(openapi_spec, path, info):
    path_parameters = info['path_parameters']
    query_parameters = info['query_parameters']
    openapi_spec['paths'].update({
        path: {
            'get': {
                'parameters': [],
                'responses': {
                    '200': {
                        'description': 'Successful response',
                    },
                },
            },
        },
    })

    if path_parameters:
        for param_name, param_details in path_parameters.items():
            openapi_spec['paths'][path]['get']['parameters'].append({
                'name': param_name,
                'in': 'path',
                'required': True,
                'schema': {
                    'type': param_details['type'],
                },
                'description': f'The {param_name} parameter',
                'example': param_details['example']
            })

    if query_parameters:
        for param_name, param_details in query_parameters.items():
            openapi_spec['paths'][path]['get']['parameters'].append({
                'name': param_name,
                'in': 'query',
                'schema': {
                    'type': param_details['type'],
                },
                'description': f'The {param_name} parameter',
                'example': param_details['example']
            })


def generate_openapi_spec(url_json):
    openapi_spec = {
        'openapi': '3.0.0',
        'info': {
            'title': 'NFL API',
            'version': '1.0.0',
        },
        'paths': {}
    } 
    for path, info in url_json.items():
        add_api_component(openapi_spec, path, info)
    return yaml.dump(openapi_spec, default_flow_style=False)

# Example

openapi_yaml = generate_openapi_spec(url_data)
with open('openapi.yaml', 'w') as f:
    f.write(openapi_yaml)

#### 1.3.2 Put it all together

In [ ]:
from nfl_client import NFLClient
client = NFLClient()
client.download_endpoints()

### 1.2 Get team data through the teams endpoint

In [35]:
api_url = 'https://api.nfl.com/football/v2/teams?limit=100&season=2023'
request(api_url)

<Response [200]>